In [1]:
import pandas as pd


base = pd.read_csv('../DataBase/autos.csv' , encoding = 'ISO-8859-1')
previsores = base.iloc[:, 1:13].values
preco_base = base.iloc[:, 0].values


## Pré-processando os dados

In [2]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

## Excluindo carros que tem o valor menor que 10 e maior que 350.000 

In [3]:
i1 = base.loc[base.price <= 10]
base = base[base.price > 10]
base = base[base.price < 350000]

In [4]:
base.loc[pd.isnull(base['vehicleType'])] # Não há dados nulos
base.loc[pd.isnull(base['gearbox'])] 
base.loc[pd.isnull(base['model'])]
base.loc[pd.isnull(base['fuelType'])]
base.loc[pd.isnull(base['notRepairedDamage'])]

valores = {'vehicleType': 'limousine', 'gearbox': 'manuell', 'model': 'golf', 'fuelType': 'benzin', 'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

previsores = base.iloc[:, 1:13].values
classe = base.iloc[:, 0].values



In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

encoder1 = LabelEncoder()
previsores[:, 0] = encoder1.fit_transform(previsores[:, 0])
previsores[:, 1] = encoder1.fit_transform(previsores[:, 1])
previsores[:, 3] = encoder1.fit_transform(previsores[:, 3])
previsores[:, 5] = encoder1.fit_transform(previsores[:, 5])
previsores[:, 8] = encoder1.fit_transform(previsores[:, 8])
previsores[:, 9] = encoder1.fit_transform(previsores[:, 9])
previsores[:, 10] = encoder1.fit_transform(previsores[:, 10])

encoder2 = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])],remainder='passthrough')
previsores = encoder2.fit_transform(previsores).toarray()





In [ ]:
# from sklearn.model_selection import train_test_split


# previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size = 0.15, random_state = 0)


In [9]:
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasRegressor

def criar_rede():
    regressor = Sequential()
    regressor.add(Dense(units = 158, activation = 'relu', input_dim = 316))
    regressor.add(Dense(units = 158, activation = 'relu'))
    regressor.add(Dense(units = 1, activation = 'linear'))
    regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics = ['mean_absolute_error'])
    return regressor


regressor = KerasRegressor(build_fn = criar_rede, epochs = 100, batch_size = 300)
resultados = cross_val_score(estimator = regressor, X = previsores, y = preco_base, cv = 10, scoring = 'neg_mean_absolute_error') 
# media = resultados.mean()
# desvio = resultados.std()

# regressor.fit(previsores, preco_base, epochs = 100, batch_size = 300)


ValueError: Found input variables with inconsistent numbers of samples: [359291, 371528]